In [16]:
rhumba::set_channels(c("conda-forge", "default")) # Rhumba makes it easier to install packages in conda; set channels for rhumba

if (!require("psych")) rhumba::install("r-psych")
if (!require("dplyr")) rhumba::install("r-dplyr")
if (!require("ggplot2")) rhumba::install("r-ggplot2")
if (!require("car")) rhumba::install("r-car")
if (!require("janitor")) rhumba::install("r-janitor")
if (!require("readxl")) rhumba::install("r-readxl")
if (!require("remotes")) rhumba::install("r-remotes")
#if (!require("hmisc")) rhumba::install("r-hmisc")
if (!require("stargazer")) rhumba::install("r-stargazer")
if (!require("ggpubr")) rhumba::install("r-ggpubr")
if (!require("ggseg")) install.packages("ggseg")
if (!require("ggseg3d")) install.packages("ggseg3d")
if (!require("easystats")) install.packages("easystats")
if (!require("matrixStats")) install.packages("matrixStats")
if (!require("ggsegDKT")) install.packages("ggsegDKT")
if (!require("plotly")) install.packages("plotly")

library('officer')


Attaching package: ‘officer’


The following object is masked from ‘package:readxl’:

    read_xlsx




# Read in data

In [3]:
library('R.matlab')

R.matlab v3.7.0 (2022-08-25 21:52:34 UTC) successfully loaded. See ?R.matlab for help.


Attaching package: ‘R.matlab’


The following objects are masked from ‘package:base’:

    getOption, isOpen




In [4]:
data=read.csv('/dagher/dagher11/filip/MAPT_OB//data/dataset_excluded_nooutlierdexclusions.csv', header=T, quote='"')

In [28]:
results=matrix(ncol=6,nrow=length(c(
                                   grep('thickness_of',colnames(data))))) 

for (i in 1:length(c(
                    grep('thickness_of',colnames(data))))) 
{ 
    
    m1=summary(lm(data[c(
                           grep('thickness_of',colnames(data)))][[i]] ~ 
          data$body_mass_index_bmi_21001.2.0 +
          poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) +
          data$sex_31.0.0 + 
          poly(difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                   min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
          data$uk_biobank_assessment_centre_54.2.0 +
          data$qualifications_6138.2.0 + data$townsend_deprivation_index_at_recruitment_189.0.0
          #data$mean_rfmri_head_motion_averaged_across_space_and_time_points_25741.2.0 + #not used because too many missing values
          #data$mean_tfmri_head_motion_averaged_across_space_and_time_points_25742.2.0)) #not used because too many missing values
                 ))
    results[i,1]=colnames(data)[c(
                                   grep('thickness_of',colnames(data)))][i] 
    results[i,2]=m1$coefficients[2,4] # p-value 
    results[i,3]=m1$coefficients[2,3] # t-value 
    results[i,4]=m1$coefficients[2,1] # sd for 
    results[i,5]=m1$coefficients[2,2] # estimate for 
    results[i,6]=ifelse(grepl("left",results[i,1]),"left","right") #save hemisphere

}

colnames(results)=c('Parcel','p-value', 't-value', 'SD', 'estimate', 'hemisphere')

In [29]:
write.table(results,'/dagher/dagher11/filip/Obesity_maps/tables/UKBB_BMI.csv',sep=',',row.names=F,col.names=T)

In [30]:
results=as.data.frame(results)

colnames(results)=c('Parcel','p-value', 't-value', 'SD', 'estimate', 'hemisphere')

results$'p-value'=as.numeric(results$'p-value')
results$'t-value'=as.numeric(results$'t-value')
results$SD=as.numeric(results$SD)
results$estimate=as.numeric(results$estimate)
results$Parcel=c("Caudal Anterior Cingulate", "Caudal Middle Frontal",
"Cuneus", "Entorhinal",
"Fusiform", "Inferior Parietal",
"Inferior Temporal", "Isthmus Cingulate",
"Lateral Occipital", "Lateral Orbitofrontal",
"Lingual", "Medial Orbitofrontal",
"Middle Temporal", "Parahippocampal",
"Paracentral", "Pars Opercularis",
"Pars Orbitalis", "Pars Triangularis",
"Pericalcarine", "Postcentral",
"Posterior Cingulate", "Precentral",
"Precuneus", "Rostral Anterior Cingulate",
"Rostral Middle Frontal", "Superior Frontal",
"Superior Parietal", "Superior Temporal",
"Supramarginal", "Transverse Temporal",
"Insula", "Caudal Anterior Cingulate",
"Caudal Middle Frontal", "Cuneus",
"Entorhinal", "Fusiform",
"Inferior Parietal", "Inferior Temporal",
"Isthmus Cingulate", "Lateral Occipital",
"Lateral Orbitofrontal", "Lingual",
"Medial Orbitofrontal", "Middle Temporal",
"Parahippocampal", "Paracentral",
"Pars Opercularis", "Pars Orbitalis",
"Pars Triangularis", "Pericalcarine",
"Postcentral", "Posterior Cingulate",
"Precentral", "Precuneus",
"Rostral Anterior Cingulate", "Rostral Middle Frontal",
"Superior Frontal", "Superior Parietal",
"Superior Temporal", "Supramarginal",
"Transverse Temporal", "Insula")
results$hemisphere=NULL


doc <- read_docx()
results[, sapply(results, is.numeric)] <- round(results[, sapply(results, is.numeric)], 4)
doc <- doc %>%
  body_add_table(as.data.frame(results))

print(doc, target = "/dagher/dagher11/filip/Obesity_maps/tables/UKBB_BMI.docx")

In [6]:
write.table(results[,3][1:31],'/dagher/dagher11/filip/Obesity_maps/data/UKB_DKT_LH.csv',
            sep=',',row.names=F, col.names=F)
write.table(results[,3][32:62],'/dagher/dagher11/filip/Obesity_maps/data/UKB_DKT_RH.csv',
            sep=',',row.names=F, col.names=F)

In [5]:
write.table(results[,3][1:62],'/dagher/dagher11/filip/Obesity_maps/data/UKB_DKT_LHRH.csv',
            sep=',',row.names=F, col.names=F)

In [7]:
writeMat('/dagher/dagher11/filip/Obesity_maps/data/UKBB_CT_DKT_LH.mat','UKBB_CT_DKT_LH'=t(as.numeric(results[,3][1:31])))
writeMat('/dagher/dagher11/filip/Obesity_maps/data/UKBB_CT_DKT_RH.mat','UKBB_CT_DKT_RH'=t(as.numeric(results[,3][32:62])))

In [31]:
writeMat('/dagher/dagher11/filip/Obesity_maps/data/UKBB_CT_DKT_LHRH.mat','UKBB_CT_DKT_LHRH'=(as.numeric(results[,3][1:62])))
